# 🏥 Health Risk Prediction: Smoking & Drinking Analysis

Data science project to predict smoking/drinking behaviors and cardiovascular risk using medical health check-up data. Apply ML techniques for preventive healthcare interventions.

## 1. Data loading and preprocessing

## 2. EDA

## 3. Feature engineering

## 4. Data preparation for modeling

## 5. Modeling - Task 1: Smoking Status

## 6. Modeling - Task 2: Drinking Status

## 7. Model Evaluation

## 8. Insights

## 9. Conclusion